In [1]:
import scipy.stats as sps
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score
from sklearn import datasets

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings

sns.set('notebook', style='whitegrid', font_scale=1.3)

# 1. Поиск гиперпараметров

В машинном обучении существуют различные семейства моделей, выбор которых зависит от задачи, которая перед вам стоит.   

Предположим, вы выбрали семейство моделей, например, линейную регрессию. Теперь вам нужно настроить гиперпараметры данной модели. Для начала нужно определить, что в машинном обучении называют гиперпараметрами.  
  
**Гиперпараметры** &mdash; это параметры модели, которые задаются ДО ее обучения, т.е. они не являются обучаемымии параметрами и не изменяются в ходе обучения модели.  

Пример: выбор способа регуляризации в модели линейной регрессии. Строго говоря, выбор семейства моделей также является гиперпараметром. 

В данном ноутбуке мы изучим различные методы подбора гиперапараметров, затем применим их к ридж-регрессии.  

В контексте линейной регрессии мы будем работать с таким гиперпараметром, как коэффициент регуляризации. Он относится к группе гиперпараметров, ограничивающих модель. Если увеличить значение гиперпараметра такого типа, то можно снизить переобучение модели.  И, наоборот, уменьшение значений даст модели возможность лучше улавливать зависимости в данных.

## 1.1. Поиск гиперпараметров для Ridge-регрессии

Будем предсказывать цены квартир в Бостоне при помощи линейной регрессии с $l_2$-регуляризацией. Вспомним, что в Ridge-регрессии мы оптимизируем 

$$
\lVert Y - X \theta\rVert^2_2 + \alpha \cdot \lVert\theta\rVert^2_2,
$$ 

где $Y$ &mdash; истинные значения целевой переменной, $X$ &mdash; матрица "объект-признак", $\theta$ &mdash; параметр модели, $\alpha$ &mdash; параметр регуляризации.

In [2]:
boston = datasets.load_boston()  # данные о ценах квартир в Бостоне
X = pd.DataFrame(data=boston['data'], columns=boston['feature_names'])
y = boston['target']

/Users/k.a.ovcharenko/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California 

Описание датасета:

In [3]:
print(boston['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [4]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


Разберемся, что означают эти признаки:  
* `CRIM` &mdash; уровень преступности на душу населения  
* `ZN` &mdash; доля жилой земли на каждые 25 000 км$^2$
* `INDUS` &mdash; доля неторговых площадей в районе
* `CHAS` &mdash; индиактор того, что неподалеку река Чарльз
* `NOX` &mdash; концентрация оксида азота
* `RM` &mdash; среднее количество комнат в доме
* `AGE` &mdash; доля зданий, построенных до 1940 года
* `DIS` &mdash; взвешенные расстояния до пяти бостонских центров занятости
* `RAD` &mdash; индекс доступности радиальных магистралей
* `TAX` &mdash; ставка налога на полную стоимость имущества за 10 000 долларов
* `PTRATIO` &mdash; количество учеников на учителя
* `B` &mdash; доля темнокожих
* `LSTAT` &mdash; доля населения с низким статусом

Получить гиперпараметры модели можно при помощи метода `get_params()`.

In [5]:
model = Ridge()
model.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': 'deprecated',
 'positive': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

Самым важным гиперпараметром тут является `alpha` &mdash; коэффициент регуляризации. Именно его мы и будем искать при помощи техники полного перебора по сетке &mdash; `GridSearch`.  
 
Суть перебора по сетке заключается в следующем: вы задаете область значений гиперпараметров, которые хотите оптимизировать, обучаете все модели, которые возможно получить в рамках заданного пространства поиска гиперпараметров. Затем выбираете из данных моделей ту, которая дала наилучшее качество на кросс-валидации. Этой модели соответствует конкретный набор значений гиперпараметров &mdash; его мы и ищем.  

В `sklearn` вам нужно просто указать одномерную сетку отдельно для каждого из гиперпараметров, построение многомерного пространства поиска оптимальных гиперпараметров спрятано в реализации метода. Применим `GridSearch` на примере:

In [6]:
# задаем пространство поиска
parameters_grid = {
    'alpha' : np.linspace(0.00001, 2, num=1000),
}

# задаем стратегию кросс-валидации
ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)

# задаем модель
model = Ridge()

# определяем поиск по сетке 
gs = GridSearchCV(
    # модель для обучения, в нашем случае Ridge
    estimator=model,
    # сетка значений гиперпараметров
    param_grid=parameters_grid,
    # метрика качества, берем MSE
    scoring='neg_mean_squared_error',
    # GridSearch отлично параллелится, указываем количество параллельных джоб
    # -1 означает использование всех ядер
    n_jobs=-1,
    # стратегия кросс-валидации
    cv=ss,
    # сообщения с логами обучения: больше значение - больше сообщений
    verbose=10,
    # значение, присваиваемое scorer в случае ошибки при обучении
    error_score='raise'
)

Стоит отметить, что в качестве scoring мы используем `neg_mean_squared_error`. Префикс neg показывает, что мы оптимизируем $(-1) \cdot \mathrm{MSE}$. Дело в том, что оптимизации в sklearn подразумевают **максимизацию** метрики качества.

In [7]:
%%time
# выполняем поиск по сетке
gs.fit(X, y)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits
[CV 2/5; 1/1000] START alpha=1e-05..............................................
[CV 2/5; 1/1000] END .............alpha=1e-05;, score=-32.267 total time=   0.0s
[CV 5/5; 3/1000] START alpha=0.004013983983983983...............................
[CV 5/5; 3/1000] END alpha=0.004013983983983983;, score=-22.424 total time=   0.0s
[CV 2/5; 5/1000] START alpha=0.008017967967967967...............................
[CV 2/5; 5/1000] END alpha=0.008017967967967967;, score=-32.275 total time=   0.0s
[CV 1/5; 6/1000] START alpha=0.010019959959959959...............................
[CV 1/5; 6/1000] END alpha=0.010019959959959959;, score=-29.792 total time=   0.0s
[CV 2/5; 7/1000] START alpha=0.01202195195195195................................
[CV 2/5; 7/1000] END alpha=0.01202195195195195;, score=-32.278 total time=   0.0s
[CV 1/5; 9/1000] START alpha=0.016025935935935934...............................
[CV 1/5; 9/1000] END alpha=0.01602593

[CV 5/5; 57/1000] START alpha=0.11212155155155154...............................
[CV 5/5; 57/1000] END alpha=0.11212155155155154;, score=-22.399 total time=   0.0s
[CV 1/5; 58/1000] START alpha=0.11412354354354354...............................
[CV 1/5; 58/1000] END alpha=0.11412354354354354;, score=-29.892 total time=   0.0s
[CV 2/5; 58/1000] START alpha=0.11412354354354354...............................
[CV 2/5; 58/1000] END alpha=0.11412354354354354;, score=-32.368 total time=   0.0s
[CV 3/5; 58/1000] START alpha=0.11412354354354354...............................
[CV 3/5; 58/1000] END alpha=0.11412354354354354;, score=-28.285 total time=   0.0s
[CV 2/5; 68/1000] START alpha=0.13414346346346345...............................
[CV 2/5; 68/1000] END alpha=0.13414346346346345;, score=-32.384 total time=   0.0s
[CV 3/5; 68/1000] START alpha=0.13414346346346345...............................
[CV 3/5; 68/1000] END alpha=0.13414346346346345;, score=-28.288 total time=   0.0s
[CV 4/5; 68/1000

[CV 3/5; 216/1000] END alpha=0.43043827827827824;, score=-28.385 total time=   0.0s
[CV 4/5; 216/1000] START alpha=0.43043827827827824..............................
[CV 4/5; 216/1000] END alpha=0.43043827827827824;, score=-22.217 total time=   0.0s
[CV 5/5; 216/1000] START alpha=0.43043827827827824..............................
[CV 5/5; 216/1000] END alpha=0.43043827827827824;, score=-22.391 total time=   0.0s
[CV 3/5; 287/1000] START alpha=0.5725797097097096...............................
[CV 3/5; 287/1000] END alpha=0.5725797097097096;, score=-28.443 total time=   0.0s
[CV 4/5; 287/1000] START alpha=0.5725797097097096...............................
[CV 4/5; 287/1000] END alpha=0.5725797097097096;, score=-22.237 total time=   0.0s
[CV 5/5; 287/1000] START alpha=0.5725797097097096...............................
[CV 5/5; 287/1000] END alpha=0.5725797097097096;, score=-22.400 total time=   0.0s
[CV 1/5; 288/1000] START alpha=0.5745817017017016...............................
[CV 1/5; 288/

[CV 4/5; 256/1000] END alpha=0.5105179579579578;, score=-22.228 total time=   0.0s
[CV 5/5; 256/1000] START alpha=0.5105179579579578...............................
[CV 5/5; 256/1000] END alpha=0.5105179579579578;, score=-22.395 total time=   0.0s
[CV 1/5; 257/1000] START alpha=0.5125199499499499...............................
[CV 1/5; 257/1000] END alpha=0.5125199499499499;, score=-30.218 total time=   0.0s
[CV 2/5; 257/1000] START alpha=0.5125199499499499...............................
[CV 2/5; 257/1000] END alpha=0.5125199499499499;, score=-32.646 total time=   0.0s
[CV 3/5; 257/1000] START alpha=0.5125199499499499...............................
[CV 3/5; 257/1000] END alpha=0.5125199499499499;, score=-28.418 total time=   0.0s
[CV 4/5; 257/1000] START alpha=0.5125199499499499...............................
[CV 4/5; 257/1000] END alpha=0.5125199499499499;, score=-22.228 total time=   0.0s
[CV 5/5; 257/1000] START alpha=0.5125199499499499...............................
[CV 5/5; 257/100

[CV 5/5; 450/1000] END alpha=0.8989044044044043;, score=-22.428 total time=   0.0s
[CV 1/5; 451/1000] START alpha=0.9009063963963962...............................
[CV 1/5; 451/1000] END alpha=0.9009063963963962;, score=-30.455 total time=   0.0s
[CV 2/5; 451/1000] START alpha=0.9009063963963962...............................
[CV 2/5; 451/1000] END alpha=0.9009063963963962;, score=-32.837 total time=   0.0s
[CV 3/5; 451/1000] START alpha=0.9009063963963962...............................
[CV 3/5; 451/1000] END alpha=0.9009063963963962;, score=-28.574 total time=   0.0s
[CV 4/5; 451/1000] START alpha=0.9009063963963962...............................
[CV 4/5; 451/1000] END alpha=0.9009063963963962;, score=-22.296 total time=   0.0s
[CV 5/5; 451/1000] START alpha=0.9009063963963962...............................
[CV 5/5; 451/1000] END alpha=0.9009063963963962;, score=-22.428 total time=   0.0s
[CV 1/5; 452/1000] START alpha=0.9029083883883883...............................
[CV 1/5; 452/100

[CV 2/5; 492/1000] END alpha=0.9829880680680679;, score=-32.870 total time=   0.0s
[CV 3/5; 492/1000] START alpha=0.9829880680680679...............................
[CV 3/5; 492/1000] END alpha=0.9829880680680679;, score=-28.605 total time=   0.0s
[CV 4/5; 492/1000] START alpha=0.9829880680680679...............................
[CV 4/5; 492/1000] END alpha=0.9829880680680679;, score=-22.312 total time=   0.0s
[CV 5/5; 492/1000] START alpha=0.9829880680680679...............................
[CV 5/5; 492/1000] END alpha=0.9829880680680679;, score=-22.436 total time=   0.0s
[CV 1/5; 493/1000] START alpha=0.98499006006006.................................
[CV 1/5; 493/1000] END alpha=0.98499006006006;, score=-30.498 total time=   0.0s
[CV 2/5; 493/1000] START alpha=0.98499006006006.................................
[CV 2/5; 493/1000] END alpha=0.98499006006006;, score=-32.870 total time=   0.0s
[CV 3/5; 493/1000] START alpha=0.98499006006006.................................
[CV 3/5; 493/1000] E

[CV 5/5; 522/1000] START alpha=1.043047827827828................................
[CV 5/5; 522/1000] END alpha=1.043047827827828;, score=-22.441 total time=   0.0s
[CV 1/5; 523/1000] START alpha=1.04504981981982.................................
[CV 1/5; 523/1000] END alpha=1.04504981981982;, score=-30.527 total time=   0.0s
[CV 2/5; 523/1000] START alpha=1.04504981981982.................................
[CV 2/5; 523/1000] END alpha=1.04504981981982;, score=-32.893 total time=   0.0s
[CV 3/5; 523/1000] START alpha=1.04504981981982.................................
[CV 3/5; 523/1000] END alpha=1.04504981981982;, score=-28.628 total time=   0.0s
[CV 4/5; 523/1000] START alpha=1.04504981981982.................................
[CV 4/5; 523/1000] END alpha=1.04504981981982;, score=-22.324 total time=   0.0s
[CV 5/5; 523/1000] START alpha=1.04504981981982.................................
[CV 5/5; 523/1000] END alpha=1.04504981981982;, score=-22.442 total time=   0.0s
[CV 1/5; 524/1000] START al

[CV 3/5; 800/1000] END alpha=1.5996016016016015;, score=-28.807 total time=   0.0s
[CV 4/5; 800/1000] START alpha=1.5996016016016015...............................
[CV 4/5; 800/1000] END alpha=1.5996016016016015;, score=-22.430 total time=   0.0s
[CV 5/5; 800/1000] START alpha=1.5996016016016015...............................
[CV 5/5; 800/1000] END alpha=1.5996016016016015;, score=-22.485 total time=   0.0s
[CV 1/5; 801/1000] START alpha=1.6016035935935935...............................
[CV 1/5; 801/1000] END alpha=1.6016035935935935;, score=-30.747 total time=   0.0s
[CV 2/5; 801/1000] START alpha=1.6016035935935935...............................
[CV 2/5; 801/1000] END alpha=1.6016035935935935;, score=-33.058 total time=   0.0s
[CV 3/5; 801/1000] START alpha=1.6016035935935935...............................
[CV 3/5; 801/1000] END alpha=1.6016035935935935;, score=-28.807 total time=   0.0s
[CV 4/5; 801/1000] START alpha=1.6016035935935935...............................
[CV 4/5; 801/100

CPU times: user 1.61 s, sys: 242 ms, total: 1.85 s
Wall time: 6.71 s


GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=0, test_size=0.25, train_size=None),
             error_score='raise', estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-05, 2.01199199e-03, 4.01398398e-03, 6.01597598e-03,
       8.01796797e-03, 1.00199600e-02, 1.20219520e-02, 1.40239439e-02,
       1.60259359e-02, 1.80279279e-02, 2.00299199e-02, 2.20319119e-02,
       2.403390...
       1.95395418e+00, 1.95595618e+00, 1.95795817e+00, 1.95996016e+00,
       1.96196215e+00, 1.96396414e+00, 1.96596614e+00, 1.96796813e+00,
       1.96997012e+00, 1.97197211e+00, 1.97397410e+00, 1.97597610e+00,
       1.97797809e+00, 1.97998008e+00, 1.98198207e+00, 1.98398406e+00,
       1.98598606e+00, 1.98798805e+00, 1.98999004e+00, 1.99199203e+00,
       1.99399402e+00, 1.99599602e+00, 1.99799801e+00, 2.00000000e+00])},
             scoring='neg_mean_squared_error', verbose=10)

Итак, мы выполнили полный перебор по сетке. Давайте посмотрим, какие атрибуты есть у `GridSearch` и чему они равны. 

Атрибут `cv_results_` возвращает информацию о времени выполнения обучения и подробности о значениях метрик. Значения данного атрибута удобнее отображать через `pd.DataFrame`.

In [8]:
results = gs.cv_results_
pd.DataFrame(results)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006648,0.001757,0.002325,0.000121,0.00001,{'alpha': 1e-05},-29.782255,-32.267382,-28.280225,-22.238042,-22.424877,-26.998556,4.018289,1
1,0.004743,0.000839,0.002664,0.000291,0.002012,{'alpha': 0.002011991991991992},-29.784217,-32.269228,-28.280004,-22.237358,-22.424236,-26.999009,4.019338,2
2,0.005031,0.000552,0.002892,0.000371,0.004014,{'alpha': 0.004013983983983983},-29.786178,-32.271071,-28.279797,-22.236683,-22.423603,-26.999466,4.020384,3
3,0.004394,0.000152,0.002804,0.000201,0.006016,{'alpha': 0.006015975975975975},-29.788137,-32.272911,-28.279602,-22.236017,-22.422978,-26.999929,4.021426,4
4,0.004191,0.000133,0.002772,0.000181,0.008018,{'alpha': 0.008017967967967967},-29.790096,-32.274748,-28.279421,-22.235360,-22.422361,-27.000397,4.022464,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.002959,0.000146,0.001818,0.000302,1.991992,{'alpha': 1.991992032032032},-30.862732,-33.139910,-28.908917,-22.498731,-22.507605,-27.583579,4.358976,996
996,0.002615,0.000444,0.001735,0.000372,1.993994,{'alpha': 1.993994024024024},-30.863263,-33.140273,-28.909397,-22.499066,-22.507703,-27.583940,4.359077,997
997,0.002591,0.000543,0.001773,0.000336,1.995996,{'alpha': 1.995996016016016},-30.863794,-33.140635,-28.909876,-22.499400,-22.507801,-27.584301,4.359177,998
998,0.002078,0.000408,0.001304,0.000297,1.997998,{'alpha': 1.997998008008008},-30.864324,-33.140997,-28.910355,-22.499735,-22.507899,-27.584662,4.359278,999


Каждой строке в этой таблице соответствуют результаты эксперимента для конкретного значения гиперпараметра. Для поиска оптимального параметра регуляризации мы выбрали сетку из 1000 элементов, соответственно, в данной таблице 1000 строк. Также напомним, что в каждом эксперименте проводится процедура кросс-валидации, то есть модель обучается несколько раз на разных батчах. Разберемся, что в каждом столбце таблицы выше:
* `mean_fit_time` &mdash; среднее время обучения модели.
* `std_fit_time` &mdash; стандартное отклонение времени обучения модели.
* `mean_score_time` &mdash; среднее время  предсказания модели. 
* `std_score_time` &mdash; стандартное отклонение времение предсказания модели.
* `param_alpha` &mdash; значение гиперпараметра. 
* `params` &mdash; значения всех гиперпараметров в виде словаря. 
* `split0_test_score` и тд &mdash; значения метрики на каждом из этапов кросс-валидации. 
* `mean_test_score` &mdash; усредненное значение метрики на тестовых батчах.
* `std_test_score` &mdash; стандартное отклонение значений метрики на тестовых батчах.
* `rank_test_score` &mdash; ранг эксперимента. Все эксприменты отсортированы по `mean_test_score`, ранг &mdash; это место эксперимента в отсортированном списке экспериментов.

Лучшая модель:

In [9]:
gs.best_estimator_ 

Ridge(alpha=1e-05)

Значение метрики лучшей модели:


In [10]:
gs.best_score_ 

-26.998556105304555

Значения гиперпараметров лучшей модели:

In [11]:
gs.best_params_ 

{'alpha': 1e-05}

Время обучения лучшей модели в секундах:

In [12]:
gs.refit_time_ 

0.0033540725708007812

Иногда пространство гиперпараметров может быть настолько огромным, что поиск по сетке занимает значительное время. В таких случаях используют `RandomizedSearch` - случайный поиск гиперпараметров.

В случае, если есть возможность использовать `GridSearch`, лучше прибегнуть к нему, так как он гарантирует перебор всех гиперпараметров сетки, но случайный поиск является неплохой альтернативой.

Попробуем применить `RandomizedSearch` в этой же задаче. Инициализируем модель похожим образом:

In [13]:
# задаем пространство поиска
parameters_grid = {
    'alpha' : np.linspace(0.00001, 2, num=1000),
}

# задаем стратегию кросс-валидации
ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)

# задаем модель
model = Ridge()

# определяем поиск по сетке 
rs = RandomizedSearchCV(
    # модель для обучения, в нашем случае Ridge
    estimator=model,
    # распределение значений гиперпараметров
    param_distributions=parameters_grid,
    # число семплирований из распределения параметров
    n_iter=100,
    # метрика качества, берем MSE
    scoring='neg_mean_squared_error',
    # GridSearch отлично параллелится, указываем количество параллельных джоб
    # -1 означает использование всех ядер
    n_jobs=-1,
    # стратегия кросс-валидации
    cv=ss,
    # сообщения с логами обучения: больше значение - больше сообщений
    verbose=10,
    # значение, присваиваемое scorer в случае ошибки при обучении
    error_score='raise'
)

Главное отличие `RandomizedSearch` от `GridSearch` заключается в наличии параметра `n_iter`, который отвечает за число семплирований. Чем больше значение параметра, тем выше качество подбора гиперпараметров, и наоборот.

Снова запустим поиск по сетке:

In [14]:
%%time
# выполняем поиск по сетке
rs.fit(X, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 4/5; 693/1000] START alpha=1.3853884584584584...............................
[CV 4/5; 693/1000] END alpha=1.3853884584584584;, score=-22.390 total time=   0.0s
[CV 5/5; 693/1000] START alpha=1.3853884584584584...............................
[CV 5/5; 693/1000] END alpha=1.3853884584584584;, score=-22.470 total time=   0.0s
[CV 1/5; 694/1000] START alpha=1.3873904504504504...............................
[CV 1/5; 694/1000] END alpha=1.3873904504504504;, score=-30.671 total time=   0.0s
[CV 2/5; 694/1000] START alpha=1.3873904504504504...............................
[CV 2/5; 694/1000] END alpha=1.3873904504504504;, score=-33.003 total time=   0.0s
[CV 3/5; 694/1000] START alpha=1.3873904504504504...............................
[CV 3/5; 694/1000] END alpha=1.3873904504504504;, score=-28.743 total time=   0.0s
[CV 4/5; 694/1000] START alpha=1.3873904504504504...............................
[CV 4/5; 694/1000] END alpha=1.38739

[CV 2/5; 937/1000] START alpha=1.8738745045045044...............................
[CV 2/5; 937/1000] END alpha=1.8738745045045044;, score=-33.118 total time=   0.0s
[CV 3/5; 937/1000] START alpha=1.8738745045045044...............................
[CV 3/5; 937/1000] END alpha=1.8738745045045044;, score=-28.880 total time=   0.0s
[CV 1/5; 940/1000] START alpha=1.8798804804804805...............................
[CV 1/5; 940/1000] END alpha=1.8798804804804805;, score=-30.832 total time=   0.0s
[CV 2/5; 940/1000] START alpha=1.8798804804804805...............................
[CV 2/5; 940/1000] END alpha=1.8798804804804805;, score=-33.119 total time=   0.0s
[CV 3/5; 940/1000] START alpha=1.8798804804804805...............................
[CV 3/5; 940/1000] END alpha=1.8798804804804805;, score=-28.881 total time=   0.0s
[CV 1/5; 943/1000] START alpha=1.8858864564564564...............................
[CV 1/5; 943/1000] END alpha=1.8858864564564564;, score=-30.834 total time=   0.0s
[CV 2/5; 943/100

[CV 2/5; 769/1000] START alpha=1.5375398498498498...............................
[CV 2/5; 769/1000] END alpha=1.5375398498498498;, score=-33.043 total time=   0.0s
[CV 3/5; 769/1000] START alpha=1.5375398498498498...............................
[CV 3/5; 769/1000] END alpha=1.5375398498498498;, score=-28.789 total time=   0.0s
[CV 4/5; 769/1000] START alpha=1.5375398498498498...............................
[CV 4/5; 769/1000] END alpha=1.5375398498498498;, score=-22.419 total time=   0.0s
[CV 5/5; 769/1000] START alpha=1.5375398498498498...............................
[CV 5/5; 769/1000] END alpha=1.5375398498498498;, score=-22.481 total time=   0.0s
[CV 1/5; 770/1000] START alpha=1.5395418418418418...............................
[CV 1/5; 770/1000] END alpha=1.5395418418418418;, score=-30.726 total time=   0.0s
[CV 2/5; 770/1000] START alpha=1.5395418418418418...............................
[CV 2/5; 770/1000] END alpha=1.5395418418418418;, score=-33.043 total time=   0.0s
[CV 3/5; 770/100

RandomizedSearchCV(cv=ShuffleSplit(n_splits=5, random_state=0, test_size=0.25, train_size=None),
                   error_score='raise', estimator=Ridge(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'alpha': array([1.00000000e-05, 2.01199199e-03, 4.01398398e-03, 6.01597598e-03,
       8.01796797e-03, 1.00199600e-02, 1.20219520e-02, 1.40239439e-02,
       1.60259359e-02, 1.80279279e-02, 2.00299199e-...
       1.95395418e+00, 1.95595618e+00, 1.95795817e+00, 1.95996016e+00,
       1.96196215e+00, 1.96396414e+00, 1.96596614e+00, 1.96796813e+00,
       1.96997012e+00, 1.97197211e+00, 1.97397410e+00, 1.97597610e+00,
       1.97797809e+00, 1.97998008e+00, 1.98198207e+00, 1.98398406e+00,
       1.98598606e+00, 1.98798805e+00, 1.98999004e+00, 1.99199203e+00,
       1.99399402e+00, 1.99599602e+00, 1.99799801e+00, 2.00000000e+00])},
                   scoring='neg_mean_squared_error', verbose=10)

Посмотрим на таблицу результатов:

In [15]:
results = rs.cv_results_
pd.DataFrame(results)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002380,0.000184,0.001551,0.000223,0.11012,{'alpha': 0.11011955955955954},-29.888005,-32.364319,-28.283950,-22.212238,-22.399551,-27.029613,4.070563,6
1,0.002620,0.000170,0.001817,0.000084,1.277281,{'alpha': 1.2772808908908908},-30.628368,-32.970804,-28.708259,-22.369677,-22.461402,-27.427702,4.309474,64
2,0.003543,0.000585,0.002347,0.000059,0.910916,{'alpha': 0.9109163563563563},-30.459930,-32.841063,-28.578304,-22.298288,-22.429405,-27.321398,4.267555,44
3,0.003251,0.000207,0.001989,0.000237,1.017022,{'alpha': 1.017021931931932},-30.513201,-32.882671,-28.617994,-22.318968,-22.439077,-27.354382,4.281412,49
4,0.003339,0.000366,0.001970,0.000281,1.729731,{'alpha': 1.729731081081081},-30.787879,-33.087740,-28.842444,-22.453571,-22.493286,-27.532984,4.344268,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.002195,0.000283,0.001378,0.000335,1.535538,{'alpha': 1.5355378578578578},-30.724779,-33.042312,-28.788139,-22.418441,-22.480748,-27.490884,4.331061,80
96,0.001699,0.000059,0.001058,0.000031,1.41742,{'alpha': 1.4174203303303303},-30.682625,-33.011333,-28.752748,-22.396403,-22.472277,-27.463077,4.321832,73
97,0.001680,0.000050,0.001153,0.000189,0.610618,{'alpha': 0.6106175575575574},-30.284547,-32.700821,-28.458430,-22.243195,-22.402630,-27.217924,4.217586,32
98,0.001816,0.000088,0.001191,0.000116,1.575578,{'alpha': 1.5755776976976976},-30.738391,-33.052212,-28.799720,-22.425799,-22.483473,-27.499919,4.333972,82


Лучшая модель:

In [16]:
rs.best_estimator_ 

Ridge(alpha=0.016025935935935934)

Значение метрики лучшей модели:


In [17]:
rs.best_score_ 

-27.00231736126069

Значения гиперпараметров лучшей модели:

In [18]:
rs.best_params_ 

{'alpha': 0.016025935935935934}

Время обучения лучшей модели в секундах:

In [20]:
rs.refit_time_ 

0.0014958381652832031